In [363]:
# Cargamos librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [364]:
#Carga desde un archivo .xlsx sin indice
Credicel = pd.read_excel('CredicelModel.xlsx')
Credicel.head(5)

,folio,folio_solicitud,precio,enganche,descuento,monto_accesorios,status,fraude,inversion,pagos_realizados,...,fecha_pago_proximo,status_cuenta,razones_buro,cp_cliente,cd_cliente,edo_cliente,cd_venta,edo_venta,curp,riesgo_cat
0,3,30,1949.0,780.0,0.0,0.0,2,0,0,0,...,Desconocido,Aun no se determina status,Desconocido,Desconocido,0,0,Tantoyuca,Veracruz,AEVM980413MVZNDR02,No hay Riesgo
1,4,33,2999.0,1050.0,0.0,0.0,2,0,0,0,...,Desconocido,Aun no se determina status,Desconocido,Desconocido,PUEBLA,PUE.,Heroica Puebla de Zaragoza,Puebla,ZALC610119MPLPRR07,No hay Riesgo
2,5,37,1959.0,490.0,0.0,0.0,3,0,0,0,...,Desconocido,Cancelado,Desconocido,Desconocido,TANTOYUCA,VER.,0,0,0,No hay Riesgo
3,6,38,1959.0,490.0,0.0,0.0,2,0,0,0,...,Desconocido,Aun no se determina status,Desconocido,Desconocido,AMXTLAN PUE,Desconocido,Zacatlan,Puebla,SAMJ790115MPLNRS02,No hay Riesgo
4,7,44,4299.0,1075.0,0.0,0.0,1,0,0,0,...,2022-03-12 00:00:00,Mora,Desconocido,Desconocido,TANTOYUCA,VER.,Tantoyuca,Veracruz,TEHF880707HVZRRR07,No hay Riesgo


In [148]:
#Carga desde un archivo .xlsx sin indice
Credicel2 = pd.read_excel('CredicelModel2.xlsx')
Credicel2.head(5)

,folio,folio_solicitud,precio,enganche,descuento,monto_accesorios,status,fraude,inversion,pagos_realizados,...,fecha_pago_proximo,status_cuenta,razones_buro,cp_cliente,cd_cliente,edo_cliente,cd_venta,edo_venta,curp,riesgo_cat
0,3,30,1949.0,780.0,0.0,0.0,2,0,0,0,...,Desconocido,Aun no se determina status,Desconocido,Desconocido,0,0,Tantoyuca,Veracruz,AEVM980413MVZNDR02,No hay Riesgo
1,4,33,2999.0,1050.0,0.0,0.0,2,0,0,0,...,Desconocido,Aun no se determina status,Desconocido,Desconocido,PUEBLA,PUE.,Heroica Puebla de Zaragoza,Puebla,ZALC610119MPLPRR07,No hay Riesgo
2,5,37,1959.0,490.0,0.0,0.0,3,0,0,0,...,Desconocido,Cancelado,Desconocido,Desconocido,TANTOYUCA,VER.,0,0,0,No hay Riesgo
3,6,38,1959.0,490.0,0.0,0.0,2,0,0,0,...,Desconocido,Aun no se determina status,Desconocido,Desconocido,AMXTLAN PUE,Desconocido,Zacatlan,Puebla,SAMJ790115MPLNRS02,No hay Riesgo
4,7,44,4299.0,1075.0,0.0,0.0,1,0,0,0,...,2022-03-12 00:00:00,Mora,Desconocido,Desconocido,TANTOYUCA,VER.,Tantoyuca,Veracruz,TEHF880707HVZRRR07,No hay Riesgo


In [150]:
#Corroboramos valores nulos
ValoresNulos = Credicel2.isnull().sum()
ValoresNulos

folio                 0
folio_solicitud       0
precio                0
enganche              0
descuento             0
monto_accesorios      0
status                0
fraude                0
inversion             0
pagos_realizados      0
reautorizacion        0
puntos                0
riesgo                0
score_buro            0
porc_eng              0
limite_credito        0
semana_actual         0
edad_cliente          0
costo_total           0
monto_financiado      0
semana                0
tag                   0
fecha                 0
marca                 0
modelo                0
plazo                 0
agente_venta          0
dis_venta             0
empresa               0
fecha_ultimo_pago     0
fecha_pago_proximo    0
status_cuenta         0
razones_buro          0
cp_cliente            0
cd_cliente            0
edo_cliente           0
cd_venta              0
edo_venta             0
curp                  0
riesgo_cat            0
dtype: int64

**Modelo 1**

In [486]:
# Declaramos las variables dependientes e independientes para la regresión logística

Vars_Indep = Credicel2[['edad_cliente','score_buro','status','semana','monto_financiado','limite_credito','costo_total','porc_eng']]
Var_Dep = Credicel2['riesgo_cat'] # Variable objetivo tipo Dicotómica

In [487]:
# Redefinimos las variables
X = Vars_Indep
y = Var_Dep

In [488]:
# Dividimos el conjunto de datos en la parte de entrenamiento y prueba:

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.40, random_state = None)

In [489]:
# Se escalan todos los datos (Se están normalizando las escalas)

escalar = StandardScaler()

In [490]:
# Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba,
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [491]:
# Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [492]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [493]:
# Realizamos una predicción

y_pred = algoritmo.predict(X_test)
y_pred

array(['No hay Riesgo', 'No hay Riesgo', 'Hay Riesgo', ..., 'Hay Riesgo',
       'No hay Riesgo', 'Hay Riesgo'], dtype=object)

In [494]:
# Verifico la matriz de Confusión

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)

Matriz de confusión: 
[[4348  484]
 [ 418 3844]]


In [495]:
# Calculo la precisión del modelo

from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average = 'binary', pos_label = 'No hay Riesgo')
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.8881700554528651


In [496]:
# Calculo la precisión del modelo

from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average = 'binary', pos_label = 'Hay Riesgo')
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.9122954259336971


In [497]:
# Calculo la exactitud del modelo

from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.9008137233340664


In [498]:
# Calculo la sensibilidad del modelo

from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average = "binary", pos_label = 'Hay Riesgo')
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.8998344370860927


In [499]:
# Calculo la sensibilidad del modelo

from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average = "binary", pos_label = 'No hay Riesgo')
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.9019239793524167


In [500]:
# El puntaje F1 es una combinación entre la precisión y la sensibilidad, para esto importamos
# Calculo el puntaje F1 del modelo

from sklearn.metrics import f1_score

puntajef1 = f1_score(y_test, y_pred, average = "binary", pos_label = "Hay Riesgo")
print('Puntaje F1 del modelo: ')
print(puntajef1)

Puntaje F1 del modelo: 
0.9060220879349864


**Modelo 2**

In [517]:
# Declaramos las variables dependientes e independientes para la regresión logística

Vars_Indep = Credicel2[['edad_cliente','score_buro','status','monto_financiado','limite_credito','costo_total','porc_eng']]
Var_Dep = Credicel2['riesgo_cat'] # Variable objetivo tipo Dicotómica

In [518]:
# Redefinimos las variables
X = Vars_Indep
y = Var_Dep

In [519]:
# Dividimos el conjunto de datos en la parte de entrenamiento y prueba:

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.40, random_state = None)

In [520]:
# Se escalan todos los datos (Se están normalizando las escalas)

escalar = StandardScaler()

In [521]:
# Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba,
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [522]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [523]:
# Realizamos una predicción

y_pred = algoritmo.predict(X_test)
y_pred

array(['No hay Riesgo', 'Hay Riesgo', 'No hay Riesgo', ...,
       'No hay Riesgo', 'No hay Riesgo', 'No hay Riesgo'], dtype=object)

In [524]:
# Verifico la matriz de Confusión

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)

Matriz de confusión: 
[[4302  513]
 [ 391 3888]]


In [525]:
# Calculo la precisión del modelo

from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average = 'binary', pos_label = 'No hay Riesgo')
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.8834355828220859


In [526]:
# Calculo la precisión del modelo

from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average = 'binary', pos_label = 'Hay Riesgo')
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.9166844236096313


In [527]:
# Calculo la exactitud del modelo

from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.900593798108643


In [528]:
# Calculo la sensibilidad del modelo

from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average = "binary", pos_label = 'Hay Riesgo')
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.8934579439252337


In [529]:
# Calculo la sensibilidad del modelo

from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average = "binary", pos_label = 'No hay Riesgo')
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.9086235101659266


In [530]:
# El puntaje F1 es una combinación entre la precisión y la sensibilidad, para esto importamos
# Calculo el puntaje F1 del modelo

from sklearn.metrics import f1_score

puntajef1 = f1_score(y_test, y_pred, average = "binary", pos_label = "Hay Riesgo")
print('Puntaje F1 del modelo: ')
print(puntajef1)

Puntaje F1 del modelo: 
0.9049221708035339


**Modelo 3**

In [531]:
# Declaramos las variables dependientes e independientes para la regresión logística

Vars_Indep = Credicel2[['edad_cliente','score_buro','monto_financiado','limite_credito','costo_total','porc_eng']]
Var_Dep = Credicel2['riesgo_cat'] # Variable objetivo tipo Dicotómica

In [532]:
# Redefinimos las variables
X = Vars_Indep
y = Var_Dep

In [533]:
# Dividimos el conjunto de datos en la parte de entrenamiento y prueba:

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.40, random_state = None)

In [534]:
# Se escalan todos los datos (Se están normalizando las escalas)

escalar = StandardScaler()

In [535]:
# Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba,
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [536]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [537]:
# Realizamos una predicción

y_pred = algoritmo.predict(X_test)
y_pred

array(['No hay Riesgo', 'Hay Riesgo', 'No hay Riesgo', ..., 'Hay Riesgo',
       'Hay Riesgo', 'No hay Riesgo'], dtype=object)

In [538]:
# Verifico la matriz de Confusión

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)

Matriz de confusión: 
[[4309  477]
 [ 365 3943]]


In [539]:
# Calculo la precisión del modelo

from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average = 'binary', pos_label = 'No hay Riesgo')
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.892081447963801


In [540]:
# Calculo la precisión del modelo

from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average = 'binary', pos_label = 'Hay Riesgo')
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.9219084296106119


In [541]:
# Calculo la exactitud del modelo

from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.9074114800967671


In [542]:
# Calculo la sensibilidad del modelo

from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average = "binary", pos_label = 'Hay Riesgo')
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.9003343083994986


In [546]:
# Calculo la sensibilidad del modelo

from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average = "binary", pos_label = 'No hay Riesgo')
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.9152739090064995


In [543]:
# El puntaje F1 es una combinación entre la precisión y la sensibilidad, para esto importamos
# Calculo el puntaje F1 del modelo

from sklearn.metrics import f1_score

puntajef1 = f1_score(y_test, y_pred, average = "binary", pos_label = "Hay Riesgo")
print('Puntaje F1 del modelo: ')
print(puntajef1)

Puntaje F1 del modelo: 
0.9109936575052854


In [544]:
# Verificar los valores sin repetirse de una columna

unico = np.unique(Credicel2['riesgo_cat'])
unico

array(['Hay Riesgo', 'No hay Riesgo'], dtype=object)

In [545]:
frecuencia = Credicel["riesgo_cat"].value_counts()

frecuencia

riesgo_cat
No hay Riesgo    16244
Hay Riesgo        6491
Name: count, dtype: int64